#### Handwritten Digit Recognition with Convolutional Neural Networks

This Python script utilizes Convolutional Neural Networks (CNNs) to perform handwritten digit recognition. The dataset used in this script consists of images of handwritten digits, with corresponding labels indicating the digit represented in each image.

#### Problem Description

The problem addressed in this script is that of handwritten digit recognition, where the goal is to correctly classify images of handwritten digits (0 through 9). The script preprocesses the image data, constructs a CNN model architecture, trains the model on the training dataset, and evaluates its performance. Finally, the trained model is used to make predictions on a separate test dataset, and the predictions are saved to a CSV file for submission.


In [19]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


In [20]:
# Read training and test data files
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Displaying the shape of the datasets
print("Training dataset shape:", train.shape)
print("Test dataset shape:", test.shape)

Training dataset shape: (42000, 785)
Test dataset shape: (28000, 784)


In [22]:
# Splitting the training dataset into features and labels
train_y = train["label"].astype("float32")
train_x = train.drop(["label"], axis=1).astype("int32")
test_x = test.astype("float32")
train_x.shape, train_y.shape, test_x.shape

((42000, 784), (42000,), (28000, 784))

In [23]:
# Reshaping and normalizing the features
train_x = train_x.values.reshape(-1, 28, 28, 1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1, 28, 28, 1)
test_x = test_x / 255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [24]:
# One Hot Encoding
train_y = tf.keras.utils.to_categorical(train_y, 10)
train_y.shape

(42000, 10)

In [25]:
# Defining the model architecture
model = tf.keras.models.Sequential([
    
    # Convolutional layers
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),  # 32 filters, 3x3 kernel, ReLU activation, input shape (28, 28, 1)
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),  # 32 filters, 3x3 kernel, ReLU activation
    tf.keras.layers.MaxPooling2D(2, 2),  # Max pooling with 2x2 pool size and stride 2
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),  # 64 filters, 3x3 kernel, ReLU activation, padding same
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),  # 64 filters, 3x3 kernel, ReLU activation, padding same
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling with 2x2 pool size and stride 2
    tf.keras.layers.Dropout(0.25),  # Dropout layer with dropout rate of 0.25
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),  # 64 filters, 3x3 kernel, ReLU activation, padding same
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),  # 64 filters, 3x3 kernel, ReLU activation, padding same
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # Max pooling with 2x2 pool size and stride 2
    tf.keras.layers.Dropout(0.25),  # Dropout layer with dropout rate of 0.25
    
    # Flatten layer
    tf.keras.layers.Flatten(),
    
    # Fully connected layers
    tf.keras.layers.Dense(256, activation="relu"),  # Dense layer with 256 units and ReLU activation
    tf.keras.layers.Dense(256, activation="relu"),  # Dense layer with 256 units and ReLU activation
    tf.keras.layers.Dropout(0.50),  # Dropout layer with dropout rate of 0.50
    
    # Output layer
    tf.keras.layers.Dense(10, activation="softmax")  # Dense layer with 10 units and softmax activation
])


# Model Architecture

![Model Architecture](model_architecture.png)


In [26]:
# Displaying the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                 

In [27]:
# Defining a callback to stop training when accuracy reaches a certain threshold
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") > 0.999:
            print("\nReached 99.9% accuracy, cancelling training!")
            self.model.stop_training = True

In [28]:
# Compiling the model
Optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adam"
)

model.compile(
    optimizer=Optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

In [32]:
# Training the model with callbacks
callbacks = myCallback()
model.fit(train_x, train_y, batch_size=50, epochs=20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 49s 45ms/step - loss: 0.3220 - accuracy: 0.8946
Epoch 2/20
840/840 [==============================] - 48s 57ms/step - loss: 0.0830 - accuracy: 0.9753
Epoch 3/20
840/840 [==============================] - 49s 59ms/step - loss: 0.0574 - accuracy: 0.9829
Epoch 4/20
840/840 [==============================] - 48s 57ms/step - loss: 0.0471 - accuracy: 0.9863
Epoch 5/20
840/840 [==============================] - 48s 57ms/step - loss: 0.0432 - accuracy: 0.9875
Epoch 6/20
840/840 [==============================] - 45s 53ms/step - loss: 0.0328 - accuracy: 0.9907
Epoch 7/20
840/840 [==============================] - 44s 52ms/step - loss: 0.0287 - accuracy: 0.9912
Epoch 8/20
840/840 [==============================] - 46s 55ms/step - loss: 0.0268 - accuracy: 0.9922
Epoch 9/20
840/840 [==============================] - 46s 55ms/step - loss: 0.0241 - accuracy: 0.9933
Epoch 10/20
840/840 [==============================] - 44s 53ms/step - loss: 0.022

In [33]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(train_x, train_y, verbose=0)

print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

Training Loss: 0.003328402992337942
Training Accuracy: 0.9990000128746033


In [34]:
# Making predictions on the test set
results = model.predict(test_x)
results = np.argmax(results, axis=1)

875/875 [==============================] - 17s 19ms/step


In [35]:
# Creating a submission dataframe
submission = pd.DataFrame({'ImageId': range(1, len(results) + 1), 'Label': results})

# Saving the submission dataframe to a CSV file
submission.to_csv("submission.csv", index=False)